In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error,r2_score
import mlflow
import mlflow.sklearn
import fredapi as fr
import fredapi.fred as fr
from xgboost import XGBRegressor

FRED_API_KEY = os.getenv("FRED_API_KEY")


fred = fr.Fred(FRED_API_KEY)



ffr = fred.get_series("FEDFUNDS")
ffr.name = "Fed Funds Rate"

ffr = fred.get_series("FEDFUNDS")
ffr.name = "Fed Funds Rate"

data = pd.DataFrame(ffr).dropna()
data = data.reset_index()
data['Date'] = data['index']


data.drop("index",axis=1,inplace=True)
data.head(10)




/home/nick/fed_funds_forecast/venv/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


,Fed Funds Rate,Date
0,0.80,1954-07-01
1,1.22,1954-08-01
2,1.07,1954-09-01
3,0.85,1954-10-01
4,0.83,1954-11-01
5,1.28,1954-12-01
6,1.39,1955-01-01
7,1.29,1955-02-01
8,1.35,1955-03-01
9,1.43,1955-04-01


In [2]:
X = data.drop("Fed Funds Rate",axis=1)
y = data['Fed Funds Rate']

X_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.13,random_state=42)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(x_test)


def evaluate(y_test,pred,model_name,r2,rmse,cv_scores):


    result = {
        "Model": model_name,
        "R2": r2,
        "RMSE": rmse,
        "Cross-val Scores":cv_scores.mean()
    }

    return result


model_dict = []
models = {
    "LinearRegression":LinearRegression(),
    "lasso":Lasso(),
    "ridge":Ridge(),
    "GradientBoostingRegressor":GradientBoostingRegressor(),
    "BaggingRegressor":BaggingRegressor(),
    "xBGboostRegression":XGBRegressor(),
    "randomforest":RandomForestRegressor(),
}


for model_name,model in models.items():
    model.fit(X_train_scaled,y_train)
    pred = model.predict(X_test_scaled)
    mse = root_mean_squared_error(y_test,pred)
    r2 = r2_score(y_test,pred)
    cv_scores = cross_val_score(model, X_train_scaled,y_train,cv=10,scoring="neg_mean_squared_error")
    model_results = evaluate(y_test, pred, model_name,r2,mse,cv_scores)
    model_dict.append(model_results)



df_results = pd.DataFrame(model_dict)
print(df_results.head(10))



                       Model        R2      RMSE  Cross-val Scores
0           LinearRegression  0.068586  2.998522        -11.641676
1                      lasso -0.004599  3.114099        -13.050213
2                      ridge  0.069840  2.996504        -11.641329
3  GradientBoostingRegressor  0.965586  0.576376         -0.446371
4           BaggingRegressor  0.990752  0.298785         -0.206648
5         xBGboostRegression  0.961390  0.610500         -0.291863
6               randomforest  0.991034  0.294194         -0.204016


In [3]:
#hyper-parameter tuning

mlflow.set_experiment("fed-funds-rate-gridsearch")

linearregression_params = {
'copy_X': [True,False], 
'fit_intercept': [True,False], 
'n_jobs': [1000,5000,10000], 
'positive': [True,False]}



lasso_params = {
    'alpha': [1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]
    }


ridge_params = {
    'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]
    }



gradientboosting_params = {
    'n_estimators': [50,100,200],
    'learning_rate': [0.01,0.1,0.2],
    'max_depth': [3,4,5],
    'min_samples_split': [2,5,10]
    }

bagginregression_params = {
    'n_estimators': [50,100,200],
    'max_samples' : [1.0,0.8,0.6],
    'max_features': [1.0,0.8,0.6]
}



xgboost_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [0, 0.1]
}


random_forest_params = {
    'n_estimators': [50,100,200],
    'min_samples_leaf':[1,2,4],
    'max_features': ['sqrt','log2',None],

}








models = {
    "LinearRegression":(LinearRegression(),linearregression_params),
    "lasso":(Lasso(),lasso_params),
    "ridge":(Ridge(),ridge_params),
    "GradientBoostingRegresser":(GradientBoostingRegressor(),gradientboosting_params),
    "baggingREgressor":(BaggingRegressor(),bagginregression_params),
    "xGBRegressor":(XGBRegressor(),xgboost_grid),
    "randomforestregressor":(RandomForestRegressor(),random_forest_params),
    }

best_scores = []

for model_name,(model,params) in models.items():
    grid_search = GridSearchCV(model,params,scoring="neg_root_mean_squared_error",cv=4,n_jobs=-1)
    grid_search.fit(X_train_scaled,y_train)
    print(f'Best Params for {model_name}: {grid_search.best_params_}')
    print(f'Best Score for {model_name}: {grid_search.best_score_}')
    best_scores.append({"Model":model_name,"neg_root_mean_squared_log_error":grid_search.best_score_})
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metric("best_score", grid_search.best_score_)
        mlflow.sklearn.log_model(grid_search.best_estimator_, model_name)
        best_scores.append({"Model": model_name, "Best Score": grid_search.best_score_})
        mlflow.sklearn.log_model(grid_search.best_estimator_, model_name)
        best_scores.append({"Model": model_name, "Best Score": grid_search.best_score_,"Best Params":grid_search.best_params_})


best_scores_df = pd.DataFrame(best_scores)



2025/09/05 05:37:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best Params for LinearRegression: {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1000, 'positive': False}
Best Score for LinearRegression: -3.4008840280728942


2025/09/05 05:37:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:37:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 05:37:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/home/nick/fed_funds_forecast/venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.350e+00, tolerance: 6.804e-01
  model = cd_fast.enet_coordinate_descent(
/home/nick/fed_funds_forecast/venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarn

Best Params for lasso: {'alpha': 0.001}
Best Score for lasso: -3.4008836413326518


2025/09/05 05:38:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 05:38:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

2025/09/05 05:38:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 05:38:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/home/nick/fed_funds_forecast/venv/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these feature

Best Params for xGBRegressor: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 500, 'reg_alpha': 0.1, 'reg_lambda': 0, 'subsample': 0.8}
Best Score for xGBRegressor: -0.6209462009593243


2025/09/05 05:38:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 05:38:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best Params for randomforestregressor: {'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 50}
Best Score for randomforestregressor: -0.4905534238340443


2025/09/05 05:38:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/05 05:38:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 05:38:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [4]:
Best_Model = RandomForestRegressor(max_features="log2",min_samples_leaf=1,n_estimators=50).fit(X_train_scaled,y_train)

pred = Best_Model.predict(X_test_scaled)
print(f'R2 Score From Best Model and HyperParameter Tuning: {r2_score(y_test,pred)*100:.2f}%')
print(f'Root Mean Squared Error from Best Model with Optimized HyperParamters: {root_mean_squared_error(y_test,pred)}')

R2 Score From Best Model and HyperParameter Tuning: 99.02%
Root Mean Squared Error from Best Model with Optimized HyperParamters: 0.3077749264594874


In [5]:
pred_vs_actual = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': pred
}, index=y_test.index)




pred_vs_actual.sort_index(inplace=True)
print('Predicted Vs Actual Prices\n')
print(pred_vs_actual)

print(pred_vs_actual.head(20))
print(pred_vs_actual.tail(20))

Predicted Vs Actual Prices

     Actual  Predicted
23     2.71     2.7458
30     2.84     2.9378
31     3.00     2.9500
33     3.00     2.9906
39     3.50     3.3796
..      ...        ...
833    5.33     5.3300
835    5.33     5.3300
840    5.33     5.3200
849    4.33     4.3330
850    4.33     4.3300

[112 rows x 2 columns]
     Actual  Predicted
23     2.71     2.7458
30     2.84     2.9378
31     3.00     2.9500
33     3.00     2.9906
39     3.50     3.3796
49     1.53     1.5008
63     3.98     3.7330
65     3.99     3.8876
66     3.99     3.8870
67     3.97     3.8684
72     3.23     3.2980
76     2.44     2.5478
77     1.98     2.4314
78     1.45     2.3782
86     1.88     2.1912
96     2.71     2.7476
109    3.49     3.0694
110    3.48     3.4352
120    3.42     3.4962
136    4.10     4.0940
     Actual  Predicted
706    0.11     0.1390
709    0.08     0.0888
713    0.09     0.0778
733    0.14     0.1350
740    0.36     0.3730
746    0.40     0.4062
753    0.90     0.8562
767  